# 環境準備

In [9]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [10]:
!pwd
%cd /content/drive/My Drive/キーフレーズ抽出
!ls

/content
/content/drive/My Drive/キーフレーズ抽出
yahoo_shopping.ipynb  花安勇人.ipynb


In [11]:
!pip install -U ginza

import pkg_resources, imp
imp.reload(pkg_resources)

import spacy
nlp = spacy.load('ja_ginza')

!pip list | grep ginza

     |████████████████████████████████| 10.4MB 6.4MB/s 
     |████████████████████████████████| 51.5MB 68kB/s 
     |████████████████████████████████| 71kB 7.6MB/s 
     |████████████████████████████████| 1.1MB 39.8MB/s 
     |████████████████████████████████| 481kB 43.3MB/s 
  Created wheel for ginza: filename=ginza-4.0.5-cp36-none-any.whl size=15898 sha256=1a600d097222dbb5a0ba552f5bdf44d1b24f064ee5262fb329f8e3c994471a63
  Stored in directory: /root/.cache/pip/wheels/a2/4f/b8/74029780ece4fedfa1f16ba6b9b86fdeee8260ade265e3756e
  Created wheel for ja-ginza: filename=ja_ginza-4.0.0-cp36-none-any.whl size=51530814 sha256=664c63044b11289ed61ea33e347ad00733e18f6635f23427093b481431dd6d07
  Stored in directory: /root/.cache/pip/wheels/28/5a/c0/95ac590b39eff99c77d729f284341d15a6903e4011d70ff421
  Created wheel for SudachiPy: filename=SudachiPy-0.4.9-cp36-cp36m-linux_x86_64.whl size=870416 sha256=543eba17ebb6926bb61dd273c5d4b85b2f72040ba757e3357ae83327782e9c16
  Stored in directory: /root/.cach

In [12]:
!pip install git+https://github.com/boudinfl/pke.git

  Cloning https://github.com/boudinfl/pke.git to /tmp/pip-req-build-dgu4uuwy
  Running command git clone -q https://github.com/boudinfl/pke.git /tmp/pip-req-build-dgu4uuwy
     |████████████████████████████████| 245kB 4.3MB/s 
  Created wheel for pke: filename=pke-1.8.1-cp36-none-any.whl size=8763598 sha256=a39e49d1ea7bf8536c1eae9b84155851301c5c9fca093e904ed8778964c26ddd
  Stored in directory: /tmp/pip-ephem-wheel-cache-r9dv3xis/wheels/8d/24/54/6582e854e9e32dd6c632af6762b3a5d2f6b181c2992e165462
Successfully built pke


In [13]:
# pkeのストップワードの定義
!python -m nltk.downloader stopwords

import nltk
import pke

pke.base.lang_stopwords['ja_ginza'] = 'japanese'
stopwords = list(list(spacy.lang.ja.stop_words.STOP_WORDS))
nltk.corpus.stopwords.words_org = nltk.corpus.stopwords.words
nltk.corpus.stopwords.words = lambda lang : stopwords if lang == 'japanese' else nltk.corpus.stopwords.words_org(lang)

/usr/lib/python3.6/runpy.py:125: RuntimeWarning: 'nltk.downloader' found in sys.modules after import of package 'nltk', but prior to execution of 'nltk.downloader'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


# テスト

# レビューの取得

In [14]:
import requests
import pprint
import json

In [15]:
url = "https://shopping.yahooapis.jp/ShoppingWebService/V1/json/reviewSearch"
api_id = "dj00aiZpPVpWS2ZXcTZpSlNQOSZzPWNvbnN1bWVyc2VjcmV0Jng9ODE-"
category_num = "38338"
offset_num = "0"

In [156]:
num_request = 50
json_list = []
start = 1

while(True):
    response = requests.get(url, params = {'appid':api_id, 'jan':4903301293255, 'results':num_request, 'start':start})
    json_data = response.json()['ResultSet']['Result']
    json_list += json_data
    if len(json_data) < num_request:
        break
    start += num_request

pprint.pprint(json_list)

[{'Count': {'All': '53'},
  'Description': '普通のナノックスから切り替えてからは、ずっと使いつづけています。柔軟剤は使ったり使わなかったり…。消臭効果も高いし、香りが好きなのでよっぽど良い商品が出なければリピートすると思います。今回は安く買えて本当に良かったです。',
  'Purpose': '',
  'Ratings': {'Average': '4.79', 'Rate': '5.00'},
  'Recommend': '0',
  'ReviewTitle': '普通のナノックスから切り替えてからは、…',
  'ReviewType': 'buyer',
  'SendTo': '',
  'Target': {'Code': 'y-lohaco_x699692',
             'Condition': 'new',
             'Image': {'Id': 'y-lohaco_x699692',
                       'Medium': {'Height': '146',
                                  'Url': 'https://item-shopping.c.yimg.jp/i/g/y-lohaco_x699692',
                                  'Width': '146'},
                       'Small': {'Height': '76',
                                 'Url': 'https://item-shopping.c.yimg.jp/i/c/y-lohaco_x699692',
                                 'Width': '76'}},
             'Name': 'セール トップ スーパーNANOX（ナノックス） ニオイ専用 詰め替え 超特大 1230g 1セット（5個入） '
                     '衣料用洗剤 ライオン',
             'Store': {'Id': 'y-lohaco',
 

# データの成型

In [157]:
import pandas as pd

In [158]:
data_list = []

for data in json_list:
    data_list.append([data['ReviewTitle'], data['Description'], data['Ratings']['Rate']])

df = pd.DataFrame(data_list, columns=['ReviewTitle', 'Description', 'Rate'])
df.head()

,ReviewTitle,Description,Rate
0,普通のナノックスから切り替えてからは、…,普通のナノックスから切り替えてからは、ずっと使いつづけています。柔軟剤は使ったり使わなかった...,5.00
1,浴室乾燥,単身赴任のために浴室乾燥機で干すことが多いためにこの商品を購入しました。香りもとてもいいと思...,5.00
2,セールの時に購入しました。使う量が今ま…,セールの時に購入しました。使う量が今までの液体洗剤より少なくてすむのでとても長持ちしそうです...,4.00
3,子供がちいさいので、毎日何度も洗濯をし…,子供がちいさいので、毎日何度も洗濯をしています。こちらは大容量なので、とても助かります。よご...,5.00
4,脱無臭化にしては、匂い残りもある気がし…,脱無臭化にしては、匂い残りもある気がします、,4.00


# レビューからキーフレーズ抽出

## キーフレーズのスコアの算出

In [159]:
def union_text_score(data, model="MultipartiteRank"):
    score_list = []
    text = ""
    for index, row in df.iterrows():
        text += row['ReviewTitle'] + " ; " + row['Description'] + ":\n"

    # モデルの定義
    if model == "MultipartiteRank": extractor = pke.unsupervised.MultipartiteRank()
    elif model == "TopicRank": extractor = pke.unsupervised.TopicRank()

    # cat_extractor = pke.unsupervised.MultipartiteRank()

    doc_text = nlp(text)
    cat_stopwords = []

    # 入力データの定義
    extractor.load_document(input=text, language='ja_ginza', normalization=None)
    
    # キーフレーズの選択（
    # 品詞体系 https://spacy.io/api/annotation#pos-tagging
    extractor.candidate_selection(pos={'NOUN', 'PROPN', 'NUM'}, stoplist=stopwords+cat_stopwords)

    # adj_list = []
    # for key in extractor.candidates.keys():
    #     key_nlp = nlp(key)
    #     if len(key_nlp) == 1 and key_nlp[0].pos_ == 'ADJ':
    #         adj_list.append(key)
    
    # print(dir(extractor.candidates))

    
    # スコアの計算
    extractor.candidate_weighting(threshold=0.5, method='average')

    return (text, extractor.get_n_best(n=10))

In [160]:
text, score_list = union_text_score(data=df, model="MultipartiteRank")
# print(text)
print(score_list)

[('今まで', 0.06739045800340081), ('におい 落ち', 0.06104439087862687), ('ナノックス', 0.05392176498177988), ('浴室 乾燥', 0.0538090303407652), ('洗濯物', 0.04354758860021986), ('漂白剤', 0.04053807743915694), ('セール', 0.039694298049812476), ('におい', 0.039224161539224825), ('液体 洗剤', 0.03913698391852467), ('汚れ落ち', 0.038945034671114084)]


In [161]:
key_list = [k[0].replace(' ', '') for k in score_list]
value_list = [[0, 0] for k in range(len(key_list))]
key_data_dict = dict(zip(key_list, value_list))

for index, row in df.iterrows():
    for key in key_list:
        if key in row['ReviewTitle']+row['Description']:
            # print(row['ReviewTitle']+row['Description'])
            key_data_dict[key][0] += float(row['Rate'])
            key_data_dict[key][1] += 1

print(key_data_dict)

{'今まで': [9.0, 2], 'におい落ち': [4.0, 1], 'ナノックス': [20.0, 4], '浴室乾燥': [5.0, 1], '洗濯物': [4.0, 1], '漂白剤': [4.0, 1], 'セール': [4.0, 1], 'におい': [9.0, 2], '液体洗剤': [4.0, 1], '汚れ落ち': [9.0, 2]}
